In [1]:
import numpy as np
import string


In [2]:
np.random.seed(1234)

In [8]:
initial={}  #start of phrase
first_order={} #second word only
second_order={}


In [12]:
def remove_punc(s):
  return s.translate(str.maketrans('','', string.punctuation))

In [6]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2023-07-25 01:14:31--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56286 (55K) [text/plain]
Saving to: ‘robert_frost.txt’

robert_frost.txt    100%[===================>]  54.97K  --.-KB/s    in 0.006s  

2023-07-25 01:14:31 (8.35 MB/s) - ‘robert_frost.txt’ saved [56286/56286]



In [7]:
def add2dict(d,k,v):
  if k not in d:
    d[k]=[]
    d[k].append(v)


In [20]:
for line in open("robert_frost.txt"):
  tokens = remove_punc(line.rstrip().lower()).split()

  T=len(tokens)
  for i in range(T):
    t=tokens[i]
    if i==0:
      #measure the distribution of the first word
      initial[t] = initial.get(t,0.) +1
    else:
      t_1=tokens[i-1]
      if i==(T-1):
        #measure probability of ending the line
        add2dict(second_order,(t_1,t),'END')
      if i==1:
        #measure distribution of second word
        #given only first word
        add2dict(first_order,t_1,t)
      else:
        t_2=tokens[i-2]
        add2dict(second_order,(t_2,t_1), t)




In [21]:
#normalize the distribtuion
initial_total=sum(initial.values())
for t,c in initial.items():
  initial[t]=c/initial_total

In [22]:
#convert [cat,cat,dog,dog,dog,mouse,....,cat] etc.
#into {cat:0.5, dog:0.2} dict of probabilities
def list2pdict(ts):
  d={}
  n=len(ts)
  for t in ts:
    d[t]=d.get(t,0.)+1
  for t,c in d.items():
    d[t]=c/n
  return d



In [23]:
for t_1,ts in first_order.items():
  #replace dictionary with list of probabilities
  first_order[t_1]=list2pdict(ts)


In [24]:
for k, ts in second_order.items():
  second_order[k]=list2pdict(ts)


In [25]:
def sample_word(d):
  #print "d:",d
  p0=np.random.random()
  #print "p0:",p0
  cumulative =0
  for t ,p in d.items():
    cumulative+=p
    if p0<cumulative:
      return t
  assert(False) #should never get here


In [26]:
def generate():
  for i in range(4): #genrate 4 lines
    sentence=[]

    #initial word
    w0=sample_word(initial)
    sentence.append(w0)

    #sample second word
    w1=sample_word(first_order[w0])
    sentence.append(w1)

    #second-order transition until END
    while True:
      w2=sample_word(second_order[(w0,w1)])
      if w2=='END':
        break
      sentence.append(w2)
      w0=w1
      w1=w2
    print(" ".join(sentence))


In [28]:
generate()

and sorry i could not travel both
the darkest evening of the year
under the shelter of the year
by picking the faded blue
